In [ ]:
#Importando e instalando as bibliotecas necessárias
!pip install pandas-profiling
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas_profiling
import seaborn as sns
%matplotlib inline

In [ ]:
#Carregando o dataset
df = pd.read_csv('../input/forest-fires-in-brazil/amazon.csv', encoding='latin1')
df.head()

<h2>Dicionário</h2>
<h3>year</h3>Ano onde ocorreu ou não incêncio.
<h3>state</h3>Estado onde ocorreu ou não incêndio.
<h3>month</h3>Mês onde ocorreu ou não incêncio.
<h3>number</h3>Número de casos
<h3>date</h3>Data dos dados analizados, armazenados no formado ano/mês/dia

<h3>Otimizando Espaço</h3>
Quando um dataset é carregado pelo pandas é atribuido um tipo de variável a cada coluna carregada, muitas vezes de forma automática é alocado um espaço maior na memória para estes valores, assim como tipos de variáveis que não correspondem as mesmas. Vamos analisar :)

In [ ]:
df.info()

In [ ]:
#Convertendo os tipos de dados
df.year = df.year.astype('int32')
df.state = df.state.astype('category')
df.month = df.month.astype('category')
df.number = df.number.astype('float64') 

In [ ]:
df.info()

A otimização do dados passou de 252.2KB para 140.0KB, assim o espaço em memória tem um melhor aproveitamento, com otimização do espaço em 55,51%

In [ ]:
#Analizando os dados gerais do dataframe
pandas_profiling.ProfileReport(df)

Com o pandas profiling é possível identificar as principais características deste dataframe, como podemos observar não há valores faltantes. Existem 5 colunas e 6454 linhas, há  617 zeros (9.6%) na coluna number, porém são os casos onde não houveram incêndio

In [ ]:
#Agrupando os estados e calculando a média de casos
df.groupby('state')['number'].mean().nlargest(27)

De 1998 a 2017 São Paulo teve uma média de casos de incêndio superior a todos os outros estados

In [ ]:
#Criando uma dataframe com a junção dos valores de 'year', 'state' e 'month', fazendo isso é preciso resetar o index.
year_state_month = df.groupby(by = ['year','state', 'month']).sum().reset_index()

In [ ]:
year_state_month.head()

In [ ]:
plt.figure(figsize=(12,4))

sns.boxplot(x = 'state', order = ['Sao Paulo', 'Mato Grosso', 'Bahia','Piau','Maranhao','Amazonas','Pará','Tocantins','Rondonia','Acre'], 
            y = 'number', data = year_state_month)

plt.title('BoxPlot Incêncio por Estado', fontsize = 18)
plt.xlabel('Estado', fontsize = 14)
plt.ylabel('Número de casos', fontsize = 14)

Embora o número médio de incêndios tenha sido maior em São Paulo (devido os outliers) o Estado do Mato Grosso apresenta uma maior variabilidade dos dados, ou seja, há um número maior de incêndios no Mato Grosso em relação a São Paulo. É possível verificar também que nos estados de São Paulo, Piaui, Amazonas a mediana está muito próxima do primeiro quartil, oque representa que os dados tem uma assimetria positiva devido aos valores discrepantes (outliers). Esta mesma característica é possível identificar nos outros estados, onde está localizada a maior parte do desmatamento florestal da Amazônia, os dados tem uma menor variabilidade e não tendem a grandes casos de incêndio, porém em casos isolados há grandes picos de incêndios com valores discrepantes igualmente críticos.

![](https://ipam.org.br/wp-content/uploads/2016/12/prodes2016.png)

In [ ]:
#Número de incêndios por ano
fires_year = df.groupby('year')['number'].sum().reset_index()
fires_year.groupby

In [ ]:
sns.set_style('whitegrid')

from matplotlib.pyplot import MaxNLocator

plt.figure(figsize=(12,4))

ax = sns.lineplot(x = 'year', y = 'number', data = year_state_month, estimator = 'sum', color = 'purple', lw = 2, 
                  err_style = None)

plt.title('Total de incêndios no Brasil : 1998 - 2017', fontsize = 18)
plt.xlabel('Ano', fontsize = 14)
plt.ylabel('Número de incêndios', fontsize = 14)

ax.xaxis.set_major_locator(plt.MaxNLocator(19))
ax.set_xlim(1998, 2017)